# Boston Housing Codeathon
Nikki Aaron

In [1]:
import pandas as pd

In [3]:
housing = pd.read_table('housing.csv', header=None) 
housing = pd.DataFrame(housing[0].str.split().to_list(), columns=['CRIM','ZN','INDUS','CHAS','NOX', 'RM','AGE','DIS','RAD','TAX', 'PTRATIO','MEDV','B','LSTAT']) 
housing = housing.astype('float32') 
housing.head() 

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,MEDV,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.199997,4.0900,1.0,296.0,15.300000,396.899994,4.98,24.000000
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.900002,4.9671,2.0,242.0,17.799999,396.899994,9.14,21.600000
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.099998,4.9671,2.0,242.0,17.799999,392.829987,4.03,34.700001
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.799999,6.0622,3.0,222.0,18.700001,394.630005,2.94,33.400002
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.200001,6.0622,3.0,222.0,18.700001,396.899994,5.33,36.200001
